In [ ]:
#!grep "E06000046" dft-road-casualty-statistics-collision-2024.csv > iw_dft-road-casualty-statistics-collision-2024.csv

In [33]:
# iw_dft-road-casualty-statistics-collision-2024

import csv
import json


def csv_to_geojson(csv_file, geojson_file):
    features = []

    with open(csv_file, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                lon = float(row["longitude"])
                lat = float(row["latitude"])
            except (ValueError, KeyError):
                continue  # skip rows with invalid coordinates

            # Build GeoJSON feature
            feature = {
                "type": "Feature",
                "geometry": {"type": "Point", "coordinates": [lon, lat]},
                "properties": {
                    k: v for k, v in row.items() if k not in ["longitude", "latitude"]
                },
            }
            features.append(feature)

    # Build FeatureCollection
    geojson = {"type": "FeatureCollection", "features": features}

    # Write output
    with open(geojson_file, "w", encoding="utf-8") as f:
        json.dump(geojson, f, indent=2)

    print(f"✅ GeoJSON saved to {geojson_file}")


csv_to_geojson(
    "iw_dft-road-casualty-statistics-collision-2024.csv", "collisions.geojson"
)

✅ GeoJSON saved to collisions.geojson


In [34]:
import folium
import json

# Load your GeoJSON file
with open("collisions.geojson", "r", encoding="utf-8") as f:
    data = json.load(f)

# Center map roughly around the first point
first_feature = data["features"][0]
lon, lat = first_feature["geometry"]["coordinates"]
m = folium.Map(location=[lat, lon], zoom_start=12)

# Add GeoJSON layer
folium.GeoJson(
    data,
    name="Collisions",
    tooltip=folium.GeoJsonTooltip(
        fields=["collision_index", "date", "time", "collision_severity"]
    ),
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

m

In [73]:
from sqlite_utils import Database

# Create database connection
db = Database("road_casualty_data.db")

# Define the table schema
db["casualties"].create(
    {
        "collision_index": str,
        "collision_year": int,
        "collision_ref_no": str,  # string (TEXT)
        "vehicle_reference": int,
        "casualty_reference": int,
        "casualty_class": int,
        "sex_of_casualty": int,
        "age_of_casualty": int,
        "age_band_of_casualty": int,
        "casualty_severity": int,
        "pedestrian_location": int,
        "pedestrian_movement": int,
        "car_passenger": int,
        "bus_or_coach_passenger": int,
        "pedestrian_road_maintenance_worker": int,
        "casualty_type": int,
        "casualty_imd_decile": int,
        "lsoa_of_casualty": str,  # string (TEXT)
        "enhanced_casualty_severity": int,
        "casualty_injury_based": int,
        "casualty_adjusted_severity_serious": int,
        "casualty_adjusted_severity_slight": int,
        "casualty_distance_banding": int,
    },
    pk=["collision_index", "casualty_reference"],
    if_not_exists=True,
)

db["collisions"].create(
    {
        "collision_index": str,
        "collision_year": int,
        "collision_ref_no": str,
        "location_easting_osgr": int,
        "location_northing_osgr": int,
        "longitude": float,
        "latitude": float,
        "police_force": int,
        "collision_severity": int,
        "number_of_vehicles": int,
        "number_of_casualties": int,
        "date": str,  # keep as TEXT (SQLite doesn’t have native date type)
        "day_of_week": int,
        "time": str,  # also store as TEXT
        "local_authority_district": str,
        "local_authority_ons_district": str,
        "local_authority_highway": str,
        "local_authority_highway_current": int,
        "first_road_class": int,
        "first_road_number": int,
        "road_type": int,
        "speed_limit": int,
        "junction_detail_historic": int,
        "junction_detail": int,
        "junction_control": int,
        "second_road_class": int,
        "second_road_number": int,
        "pedestrian_crossing_human_control_historic": int,
        "pedestrian_crossing_physical_facilities_historic": int,
        "pedestrian_crossing": int,
        "light_conditions": int,
        "weather_conditions": int,
        "road_surface_conditions": int,
        "special_conditions_at_site": int,
        "carriageway_hazards_historic": int,
        "carriageway_hazards": int,
        "urban_or_rural_area": int,
        "did_police_officer_attend_scene_of_accident": int,
        "trunk_road_flag": int,
        "lsoa_of_accident_location": str,
        "enhanced_severity_collision": int,
        "collision_injury_based": int,
        "collision_adjusted_severity_serious": float,
        "collision_adjusted_severity_slight": float,
    },
    pk="collision_index",
    if_not_exists=True,
)

db["vehicles"].create(
    {
        "collision_index": str,
        "collision_year": int,
        "collision_ref_no": str,
        "vehicle_reference": int,
        "vehicle_type": int,
        "towing_and_articulation": int,
        "vehicle_manoeuvre_historic": int,
        "vehicle_manoeuvre": int,
        "vehicle_direction_from": int,
        "vehicle_direction_to": int,
        "vehicle_location_restricted_lane_historic": int,
        "vehicle_location_restricted_lane": int,
        "junction_location": int,
        "skidding_and_overturning": int,
        "hit_object_in_carriageway": int,
        "vehicle_leaving_carriageway": int,
        "hit_object_off_carriageway": int,
        "first_point_of_impact": int,
        "vehicle_left_hand_drive": int,
        "journey_purpose_of_driver_historic": int,
        "journey_purpose_of_driver": int,
        "sex_of_driver": int,
        "age_of_driver": int,
        "age_band_of_driver": int,
        "engine_capacity_cc": int,
        "propulsion_code": int,
        "age_of_vehicle": int,
        "generic_make_model": str,
        "driver_imd_decile": int,
        "lsoa_of_driver": str,
        "escooter_flag": int,
        "driver_distance_banding": int,
    },
    pk=("collision_index", "vehicle_reference"),
    if_not_exists=True,
)

<Table vehicles (collision_index, collision_year, collision_ref_no, vehicle_reference, vehicle_type, towing_and_articulation, vehicle_manoeuvre_historic, vehicle_manoeuvre, vehicle_direction_from, vehicle_direction_to, vehicle_location_restricted_lane_historic, vehicle_location_restricted_lane, junction_location, skidding_and_overturning, hit_object_in_carriageway, vehicle_leaving_carriageway, hit_object_off_carriageway, first_point_of_impact, vehicle_left_hand_drive, journey_purpose_of_driver_historic, journey_purpose_of_driver, sex_of_driver, age_of_driver, age_band_of_driver, engine_capacity_cc, propulsion_code, age_of_vehicle, generic_make_model, driver_imd_decile, lsoa_of_driver, escooter_flag, driver_distance_banding)>

In [74]:
# Install sqlite-utils if not already installed
# !pip install sqlite-utils
import pandas as pd

# Load CSV files into SQLite tables
print("Loading CSV files into SQLite database...")

# Load vehicle data
print("Loading vehicle data...")

db["vehicles"].insert_all(
    pd.read_csv("dft-road-casualty-statistics-vehicle-2024.csv").to_dict("records")
)

# Load casualty data
print("Loading casualty data...")
db["casualties"].insert_all(
    pd.read_csv("dft-road-casualty-statistics-casualty-2024.csv").to_dict("records")
)


# Load collision data
print("Loading collision data...")
db["collisions"].insert_all(
    pd.read_csv("dft-road-casualty-statistics-collision-2024.csv").to_dict("records")
)

print("Data loaded successfully!")

# Create indexes for better join performance
print("Creating indexes...")
db["vehicles"].create_index(["collision_index", "vehicle_reference"])
db["casualties"].create_index(["collision_index", "vehicle_reference"])
db["collisions"].create_index(["collision_index"])

Loading CSV files into SQLite database...
Loading vehicle data...


/var/folders/10/8lp40z8x2hj_cch9j4rk_cpm0000gs/T/ipykernel_4235/953079631.py:11: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  db["vehicles"].insert_all(pd.read_csv("dft-road-casualty-statistics-vehicle-2024.csv").to_dict("records"))


Loading casualty data...
Loading collision data...


/var/folders/10/8lp40z8x2hj_cch9j4rk_cpm0000gs/T/ipykernel_4235/953079631.py:21: DtypeWarning: Columns (0,2,39) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("dft-road-casualty-statistics-collision-2024.csv").to_dict("records")


Data loaded successfully!
Creating indexes...


<Table collisions (collision_index, collision_year, collision_ref_no, location_easting_osgr, location_northing_osgr, longitude, latitude, police_force, collision_severity, number_of_vehicles, number_of_casualties, date, day_of_week, time, local_authority_district, local_authority_ons_district, local_authority_highway, local_authority_highway_current, first_road_class, first_road_number, road_type, speed_limit, junction_detail_historic, junction_detail, junction_control, second_road_class, second_road_number, pedestrian_crossing_human_control_historic, pedestrian_crossing_physical_facilities_historic, pedestrian_crossing, light_conditions, weather_conditions, road_surface_conditions, special_conditions_at_site, carriageway_hazards_historic, carriageway_hazards, urban_or_rural_area, did_police_officer_attend_scene_of_accident, trunk_road_flag, lsoa_of_accident_location, enhanced_severity_collision, collision_injury_based, collision_adjusted_severity_serious, collision_adjusted_severity_s

In [75]:
# Example joined query - modify as needed
joined_query = """
SELECT 
    c.collision_index,
    c.collision_year,
    c.date,
    c.time,
    c.police_force,
    c.collision_severity,
    c.number_of_vehicles,
    c.number_of_casualties,
    c.longitude,
    c.latitude,
    c.speed_limit,
    c.weather_conditions,
    c.light_conditions,
    c.urban_or_rural_area,
    
    v.vehicle_reference,
    v.vehicle_type,
    v.vehicle_manoeuvre,
    v.sex_of_driver,
    v.age_of_driver,
    v.age_band_of_driver,
    v.journey_purpose_of_driver,
    v.generic_make_model,
    
    cas.casualty_reference,
    cas.casualty_class,
    cas.sex_of_casualty,
    cas.age_of_casualty,
    cas.age_band_of_casualty,
    cas.casualty_severity,
    cas.casualty_type
    
FROM collisions c
LEFT JOIN vehicles v ON c.collision_index = v.collision_index
LEFT JOIN casualties cas ON c.collision_index = cas.collision_index 
                         AND v.vehicle_reference = cas.vehicle_reference

WHERE c.local_authority_ons_district ='E06000046'
ORDER BY c.collision_index, v.vehicle_reference, cas.casualty_reference
"""

print("Executing joined query...")
result_df = pd.read_sql_query(joined_query, db.conn)

# Save to CSV
output_filename = "joined_road_casualty_data.csv"
result_df.to_csv(output_filename, index=False)
print(f"Joined data saved to {output_filename}")

print(f"Shape of joined data: {result_df.shape}")
print("\nFirst few rows:")
print(result_df.head())

# Show table info
print("\nTable information:")
for table_name in ["collisions", "vehicles", "casualties"]:
    count = db[table_name].count
    print(f"{table_name}: {count:,} rows")

# Optional: Show some sample statistics
print("\nSample statistics:")
print(f"Unique collisions: {result_df['collision_index'].nunique():,}")
print(f"Total vehicles involved: {result_df['vehicle_reference'].count():,}")
print(f"Total casualties: {result_df['casualty_reference'].count():,}")

Executing joined query...
Joined data saved to joined_road_casualty_data.csv
Shape of joined data: (447, 29)

First few rows:
  collision_index  collision_year        date   time  police_force  \
0   2024440002270            2024  02/01/2024  17:25            44   
1   2024440005303            2024  04/01/2024  17:31            44   
2   2024440005303            2024  04/01/2024  17:31            44   
3   2024440009385            2024  07/01/2024  18:18            44   
4   2024440012548            2024  09/01/2024  15:45            44   

   collision_severity  number_of_vehicles  number_of_casualties  longitude  \
0                   3                   1                     1   -1.29396   
1                   3                   2                     1   -1.14858   
2                   3                   2                     1   -1.14858   
3                   3                   1                     1   -1.26854   
4                   3                   1                     1

In [76]:
# Additional imports needed
import json
import folium
from folium import plugins
import numpy as np

# Assuming you already have result_df from the previous query
# If not, run the previous script first

# Clean the data - remove rows with invalid coordinates
print("Cleaning coordinate data...")
clean_df = result_df.dropna(subset=["longitude", "latitude"])
clean_df = clean_df[
    (clean_df["longitude"] != 0)
    & (clean_df["latitude"] != 0)
    & (clean_df["longitude"] > -10)
    & (clean_df["longitude"] < 2)  # Rough UK bounds
    & (clean_df["latitude"] > 49)
    & (clean_df["latitude"] < 61)
]

print(f"Valid coordinates: {len(clean_df):,} out of {len(result_df):,} total records")

# Create severity color mapping
severity_colors = {
    1: "red",  # Fatal
    2: "orange",  # Serious
    3: "yellow",  # Slight
}

severity_names = {1: "Fatal", 2: "Serious", 3: "Slight"}

# Generate GeoJSON
print("Generating GeoJSON...")
features = []
for idx, row in clean_df.iterrows():
    if pd.isna(row["longitude"]) or pd.isna(row["latitude"]):
        continue

    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [float(row["longitude"]), float(row["latitude"])],
        },
        "properties": {
            "collision_index": row["collision_index"],
            "date": row["date"],
            "time": row["time"],
            "collision_severity": (
                int(row["collision_severity"])
                if pd.notna(row["collision_severity"])
                else None
            ),
            "severity_name": (
                severity_names.get(int(row["collision_severity"]), "Unknown")
                if pd.notna(row["collision_severity"])
                else "Unknown"
            ),
            "number_of_vehicles": (
                int(row["number_of_vehicles"])
                if pd.notna(row["number_of_vehicles"])
                else None
            ),
            "number_of_casualties": (
                int(row["number_of_casualties"])
                if pd.notna(row["number_of_casualties"])
                else None
            ),
            "speed_limit": (
                int(row["speed_limit"]) if pd.notna(row["speed_limit"]) else None
            ),
            "weather_conditions": (
                int(row["weather_conditions"])
                if pd.notna(row["weather_conditions"])
                else None
            ),
            "light_conditions": (
                int(row["light_conditions"])
                if pd.notna(row["light_conditions"])
                else None
            ),
            "urban_or_rural_area": (
                int(row["urban_or_rural_area"])
                if pd.notna(row["urban_or_rural_area"])
                else None
            ),
        },
    }
    features.append(feature)

geojson_data = {"type": "FeatureCollection", "features": features}

# Save GeoJSON file
geojson_filename = "road_casualty_data.geojson"
with open(geojson_filename, "w") as f:
    json.dump(geojson_data, f, indent=2)
print(f"GeoJSON saved to {geojson_filename}")

# Create Leaflet map
print("Creating Leaflet map...")

# Calculate map center
center_lat = clean_df["latitude"].mean()
center_lon = clean_df["longitude"].mean()

# Create base map
m = folium.Map(location=[center_lat, center_lon], zoom_start=7, tiles="OpenStreetMap")

# Add tile layer options
folium.TileLayer("cartodbdark_matter", name="Dark Mode").add_to(m)
folium.TileLayer("cartodbpositron", name="Light Mode").add_to(m)

# Create marker clusters for better performance with large datasets
marker_cluster = plugins.MarkerCluster(
    name="All Casualties",
    options={
        "disableClusteringAtZoom": 15,  # Stop clustering at zoom level 15
        "maxClusterRadius": 50,  # Smaller cluster radius
    },
).add_to(m)

# Add markers to cluster
for idx, row in clean_df.iterrows():
    if pd.isna(row["longitude"]) or pd.isna(row["latitude"]):
        continue

    severity = (
        int(row["collision_severity"]) if pd.notna(row["collision_severity"]) else 3
    )
    color = severity_colors.get(severity, "blue")
    severity_name = severity_names.get(severity, "Unknown")

    popup_text = f"""
    <b>Collision {row['collision_index']}</b><br>
    Date: {row['date']}<br>
    Time: {row['time']}<br>
    Severity: {severity_name}<br>
    Vehicles: {int(row['number_of_vehicles']) if pd.notna(row['number_of_vehicles']) else 'Unknown'}<br>
    Casualties: {int(row['number_of_casualties']) if pd.notna(row['number_of_casualties']) else 'Unknown'}<br>
    Speed Limit: {int(row['speed_limit']) if pd.notna(row['speed_limit']) else 'Unknown'} mph
    """

    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=4,
        popup=folium.Popup(popup_text, max_width=300),
        color="black",
        weight=3,
        fillColor=color,
        fillOpacity=0.7,
    ).add_to(marker_cluster)

# Add separate layers for each severity level
for severity, color in severity_colors.items():
    severity_data = clean_df[clean_df["collision_severity"] == severity]
    if len(severity_data) > 0:
        severity_group = folium.FeatureGroup(
            name=f"{severity_names[severity]} ({len(severity_data)})"
        )

        for idx, row in severity_data.iterrows():
            folium.CircleMarker(
                location=[row["latitude"], row["longitude"]],
                radius=5,
                popup=folium.Popup(
                    f"""
                <b>{severity_names[severity]} Collision</b><br>
                Index: {row['collision_index']}<br>
                Date: {row['date']}<br>
                Time: {row['time']}<br>
                Vehicles: {int(row['number_of_vehicles']) if pd.notna(row['number_of_vehicles']) else 'Unknown'}<br>
                Casualties: {int(row['number_of_casualties']) if pd.notna(row['number_of_casualties']) else 'Unknown'}
                """,
                    max_width=300,
                ),
                color="black",
                weight=3,
                fillColor=color,
                fillOpacity=0.8,
            ).add_to(severity_group)

        severity_group.add_to(m)

# Add heatmap layer
heat_data = [
    [row["latitude"], row["longitude"]]
    for idx, row in clean_df.iterrows()
    if pd.notna(row["latitude"]) and pd.notna(row["longitude"])
]

heatmap = plugins.HeatMap(
    heat_data,
    name="Heatmap",
    radius=15,
    blur=10,
    gradient={0.2: "blue", 0.4: "purple", 0.6: "orange", 1.0: "red"},
)
heatmap.add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Add legend
legend_html = """
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 150px; height: 140px; 
            background-color: white; border:2px solid grey; z-index:9999; 
            font-size:14px; padding: 10px">
<p><b>Collision Severity</b></p>
<p><i class="fa fa-circle" style="color:red"></i> Fatal</p>
<p><i class="fa fa-circle" style="color:orange"></i> Serious</p>
<p><i class="fa fa-circle" style="color:yellow"></i> Slight</p>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# Save map
map_filename = "road_casualty_map.html"
m.save(map_filename)
print(f"Interactive map saved to {map_filename}")

# Print summary statistics
print(f"\nSummary Statistics:")
print(f"Total valid locations plotted: {len(clean_df):,}")
print(f"Severity breakdown:")
for severity in [1, 2, 3]:
    count = len(clean_df[clean_df["collision_severity"] == severity])
    print(f"  {severity_names[severity]}: {count:,}")

print(f"\nFiles generated:")
print(f"1. {geojson_filename} - GeoJSON file for use in GIS software")
print(f"2. {map_filename} - Interactive Leaflet map")

Cleaning coordinate data...
Valid coordinates: 447 out of 447 total records
Generating GeoJSON...
GeoJSON saved to road_casualty_data.geojson
Creating Leaflet map...
Interactive map saved to road_casualty_map.html

Summary Statistics:
Total valid locations plotted: 447
Severity breakdown:
  Fatal: 8
  Serious: 150
  Slight: 289

Files generated:
1. road_casualty_data.geojson - GeoJSON file for use in GIS software
2. road_casualty_map.html - Interactive Leaflet map


In [78]:
m

In [79]:
m